In [1]:
import numpy as np
import pandas as pd
from siuba import *
import altair as alt
import altair_saver
from calitp import *
from plotnine import *
import intake

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


## Read in Catalog

In [116]:
catalog = intake.open_catalog("catalog.yml")

In [120]:
#ERROR. probably because it is reading an excel sheet
#df = catalog.black_cat_5311data.read()

In [95]:
gtfs_status = catalog.gtfs_status.read()

In [99]:
gtfs_status = to_snakecase(gtfs_status)

In [102]:
gtfs_status.head()

,name,itp_id,reporting_category,caltrans_district,gtfs_static_status,gtfs_realtime_status,missing_static,services_needing_alerts,services_needing_tripupdates_or_vehiclepositions
0,Long Beach Transit,170.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,AquaLink,"Galaxy Express, AquaLink, UCLA/Westwood Commut...","Galaxy Express, AquaLink, UCLA/Westwood Commut..."
1,City of Maywood,196.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,Maywood Express Shuttle,"Maywood Dial-A-Ride, Maywood Express Shuttle","Maywood Dial-A-Ride, Maywood Express Shuttle"
2,City of Cerritos,63.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,Cerritos on Wheels,"Cerritos on Wheels, Cerritos Dial-A-Ride","Cerritos on Wheels, Cerritos Dial-A-Ride"
3,City of La Puente,152.0,Other Public Transit,07 - Los Angeles,Static Incomplete,RT Incomplete,La Puente Link,"La Puente Link, La Puente Dial-A-Ride","La Puente Link, La Puente Dial-A-Ride"
4,City of Artesia,20.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,Artesia Transit,"Artesia Express Dial-A-Ride, Artesia Transit","Artesia Express Dial-A-Ride, Artesia Transit"


In [101]:
gtfs_status.name.nunique()

190

## Read in Vehicle Data

In [18]:
vehicles =  pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020-Vehicles_1.xlsm',
                                   sheet_name = ['Age Distribution', 'Vehicle Type Count by Agency'])

In [21]:
print(type(vehicles))

<class 'dict'>


In [22]:
vehicle_age = vehicles['Age Distribution']
vehicle_count = vehicles['Vehicle Type Count by Agency']

In [27]:
vehicle_age = (vehicle_age>>filter(_.State=='CA'))

In [29]:
vehicle_age.State.unique()

array(['CA'], dtype=object)

* **note:** multiple entries for each agency

In [35]:
vehicle_count = vehicle_count>>filter(_.State=='CA')

In [49]:
vehicle_count.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 2 to 2662
Data columns (total 96 columns):
 #   Column                                                        Non-Null Count  Dtype 
---  ------                                                        --------------  ----- 
 0   Agency                                                        226 non-null    object
 1   City                                                          226 non-null    object
 2   State                                                         226 non-null    object
 3   Legacy NTD ID                                                 156 non-null    object
 4   NTD ID                                                        226 non-null    object
 5   Organization Type                                             226 non-null    object
 6   Reporter Type                                                 226 non-null    object
 7   Primary UZA Population                                        226 non-null    i

In [ ]:
vehicle_count

In [122]:
len(vehicle_age)

705

In [48]:
vehicle_count = vehicle_count.drop(columns=['Unnamed: 95', 'Unnamed: 96', 'Unnamed: 97'])

In [51]:
print(vehicle_age.Agency.nunique())
print(vehicle_count.Agency.nunique())

218
226


## Read in Agency Information

In [2]:
agency_info = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020_Agency_Information.xlsx',
                                      sheet_name=None),ignore_index=True)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [3]:
agency_info = to_snakecase(agency_info)

In [4]:
agency_info.sample(5)

,ntd_id,legacy_ntd_id,agency_name,reporter_acronym,doing_business_as,reporter_type,reporting_module,organization_type,reported_by_ntd_id,reported_by_name,...,sq_miles,voms_do,voms_pt,total_voms,volunteer_drivers,personal_vehicles,tam_tier,number_of_state_counties,number_of_counties_with_service,state_admin_funds_expended
246,30042,3042,Washington County,NaN,Washington County Transit,Reduced Reporter,Urban,"City, County or Local Government Unit or Depar...",NaN,NaN,...,133.0,12.0,NaN,12.0,NaN,NaN,Tier II,NaN,NaN,NaN
2598,8R01-80202,8R01-028,Town of Berthoud,BATS,Berthoud Area Transportation Services,Reduced Asset Reporter,Asset,"City, County or Local Government Unit or Depar...",8R01,Colorado Department of Transportation,...,NaN,NaN,NaN,NaN,NaN,NaN,Tier II,NaN,NaN,NaN
1032,90279,NaN,Los Angeles County Dept. of Public Works - Wil...,NaN,NaN,Reduced Reporter,Urban,"City, County or Local Government Unit or Depar...",NaN,NaN,...,1736.0,NaN,3.0,3.0,NaN,NaN,Tier II,NaN,NaN,NaN
342,40040,4040,Jacksonville Transportation Authority,JTA,NaN,Full Reporter,Urban,Independent Public Agency or Authority of Tran...,NaN,NaN,...,530.0,165.0,132.0,297.0,NaN,NaN,Tier I (Rail),NaN,NaN,NaN
2304,6R02-66297,NaN,ARC of St. Martin,NaN,The Arc,Reduced Asset Reporter,Asset,Private-Non-Profit Corporation,6R02,Louisiana Department of Transportation,...,NaN,NaN,NaN,NaN,NaN,NaN,Tier II,NaN,NaN,NaN


In [5]:
agency_info = agency_info>>filter(_.state=='CA')

In [6]:
print(agency_info.agency_name.nunique())

236


In [ ]:
agency_crosswalk = agency_info[['agency_name','ntd_id']]

In [ ]:
agency_crosswalk>>count(_.agency_name)>>arrange(-_.n)

In [85]:
agency_revenue = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020_revenue_sources.xlsx',
                                      sheet_name=None), ignore_index=True)

In [86]:
agency_revenue = to_snakecase(agency_revenue)

In [125]:
agency_revenue.sample(5)

,ntd_id,agency_name,reporter_type,subrecipient_type,reporting_module,funding_category,funds_expended_type,total_of_passenger_fares,park_and_ride_revenue__earned_only_,other_agency_revenue__earned_only_,...,funds_received_from_other_usdot_grant_programs,other_usdot_grants_description,other_federal_funds,other_federal_funds_description,contributed_services,voluntary_non_exchange_transactions,voluntary_non_exchange_transactions_description,sales_and_disposals_of_assets,transportation_development_credits,total
6597,40251,City of Pembroke Pines,Reduced Reporter,NaN,Urban,State Government,Funds Expended on Capital,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4711,3R06-30154,Greene Co. Transit Inc.,Rural Reporter,Rural General Public Transit,Rural,State Government,Funds Expended on Operations,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,215198
13902,6R03-60224,The Community Pantry,Rural Reporter,Rural General Public Transit,Rural,Local Government,Funds Expended on Operations,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
19473,90302,San Bernardino County Transportation Authority,Full Reporter,NaN,Urban,State Government,Funds Earned During Period,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
15054,7R02-70102,Decatur County Transportation Bus,Rural Reporter,Rural General Public Transit,Rural,Local Government,Funds Expended on Capital,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [115]:
agency_revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20446 entries, 0 to 20445
Data columns (total 93 columns):
 #   Column                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                --------------  -----  
 0   ntd_id                                                                                                                20446 non-null  object 
 1   agency_name                                                                                                           20446 non-null  object 
 2   reporter_type                                                                                                         20446 non-null  object 
 3   subrecipient_type                                                                                                     9618 non-null   object 
 4   repo

* no state information. have to merge with other dataframes to get the information we need.

## Read BlackCat Data

In [52]:
df = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /Grant_Projects.xlsx', sheet_name=None), ignore_index=True)

In [53]:
df.sample(5)

,Grant Fiscal Year,Funding Program,Grant Number,Project Year,Organization Name,UPIN,Description,ALI,Contract Number,AllocationAmount,Encumbered Amount,ExpendedAmount,ActiveBalance,ClosedOutBalance,Project Status,Project Closed By,Project Closed Date,Project Closed Time
1028,2018,Section 5311,CA-2018-114 | 0019000012,2018,Yuba-Sutter Transit Authority,BCG0001464,Purchase Replacement < 30 Ft Bus,111204,64BC19-00829,13100.20,75000.00,13100.20,0.0,0,Open,NaN,NaN,NaN
1929,2020,CARES Act,CA-2020-054 | 0020000192,2020,San Luis Obispo Regional Transit Authority,BCG0002576,Emergency Relief- Operating Assistance 100% Fe...,300908,64VO20-01066,591789.00,591789.00,591789.00,0.0,0,Open,NaN,NaN,NaN
325,2017,5310 Trad,CA-2017-169 | 0017000186-T,2017,Dignity Health Connected Living,BCG0000841,Purchase Replacement < 30 Ft Bus,111204,64AC18-00610,-6457.43,63542.57,63542.57,-70000.0,0,Open,NaN,NaN,NaN
1360,2019,5310 Trad,CA-2020-244 | 0020000273-T,2019,New Advances for People with Disabilities,BCG0002606,Purchase Replacement Van,111215,64AC19-01201,44800.00,56000.00,0.00,44800.0,0,Open,NaN,NaN,NaN
651,2017,5311(f) Cont,TBD,2017,Napa Valley Transportation Authority,BCG0000129,Operating Assistance,300902,64CO17-00471,300000.00,300000.00,300000.00,0.0,0,Open,NaN,NaN,NaN


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520 entries, 0 to 2519
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Grant Fiscal Year    2520 non-null   int64  
 1   Funding Program      2520 non-null   object 
 2   Grant Number         2520 non-null   object 
 3   Project Year         2520 non-null   int64  
 4   Organization Name    2520 non-null   object 
 5   UPIN                 2520 non-null   object 
 6   Description          2520 non-null   object 
 7   ALI                  2520 non-null   object 
 8   Contract Number      2239 non-null   object 
 9   AllocationAmount     2520 non-null   float64
 10  Encumbered Amount    2520 non-null   float64
 11  ExpendedAmount       2520 non-null   float64
 12  ActiveBalance        2520 non-null   float64
 13  ClosedOutBalance     2520 non-null   int64  
 14  Project Status       2520 non-null   object 
 15  Project Closed By    0 non-null      f

In [55]:
df = to_snakecase(df)

In [56]:
df = df.drop(columns=['project_closed_by', 'project_closed_date', 'project_closed_time'])

In [60]:
df.sample()

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status
1556,2019,5310 Trad,CA-2020-244 | 0020000273-T,2019,United Cerebral Palsy Association of Greater S...,BCG0002051,Purchase Replacement < 30 Ft Bus,111204,64AC19-01243,64960.0,84035.43,81200.0,-16240.0,0,Open


In [61]:
df.duplicated().value_counts()

False    2520
dtype: int64

In [62]:
df>>filter(_.funding_program.str.contains('5311'))>>count(_.funding_program)

,funding_program,n
0,5311(f) Cont,41
1,5311(f) Round 2,27
2,CMAQ (FTA 5311),31
3,Section 5311,590
4,Section 5311(f),109


In [63]:
df5311 = (df>>filter(_.funding_program.str.contains('5311')))

In [64]:
df5311_agency = (df5311 >> count(_.organization_name) >> arrange(-_.n))

In [65]:
print(len(df5311_agency.organization_name.unique()))

88


In [66]:
df5311_agency.head()

,organization_name,n
60,Mountain Area Regional Transit Authority,33
76,Sonoma County Transit,31
57,Modoc Transportation Agency,28
58,Monterey-Salinas Transit,28
87,Yuba-Sutter Transit Authority,28


In [67]:
sorted(df5311_agency.organization_name.unique())

['Alpine County Community Development',
 'Amador Transit',
 'Butte County Association of Governments/ Butte Regional Transit',
 'Calaveras County Public Works',
 'Calaveras Transit Agency ',
 'City of Arcata',
 'City of Arvin',
 'City of Auburn',
 'City of California City',
 'City of Chowchilla ',
 'City of Corcoran - Corcoran Area Transit',
 'City of Dinuba ',
 'City of Dixon',
 'City of Escalon ',
 'City of Fairfield',
 'City of Guadalupe',
 'City of McFarland',
 'City of Needles',
 'City of Ojai',
 'City of Porterville',
 'City of Ridgecrest',
 'City of Rio Vista',
 'City of Santa Maria',
 'City of Shafter',
 'City of Solvang',
 'City of Taft',
 'City of Tehachapi',
 'City of Visalia',
 'City of Wasco',
 'City of Woodlake',
 'Colusa County Transit Agency',
 'County Connection (Central Contra Costa Transit Authority)',
 'County of Los Angeles - Department of Public Works',
 'County of Mariposa',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Depar

* no obvious duplicates or multiple names for one agency

In [68]:
df5311 >> group_by(_.organization_name) >> summarize(n=_.ali.nunique()) >> arrange(-_.n)

,organization_name,n
46,Humboldt Transit Authority,6
18,City of Ojai,5
58,Monterey-Salinas Transit,5
12,City of Dixon,4
54,Madera County,4
...,...,...
80,Town of Truckee,1
81,Transit Joint Powers Authority for Merced County,1
83,Tuolumne County Transit Agency (TCTA),1
84,Victor Valley Transit Authority,1


In [69]:
df5311 >> group_by(_.organization_name) >> summarize(n=_.grant_number.nunique()) >> arrange(-_.n)

,organization_name,n
60,Mountain Area Regional Transit Authority,14
46,Humboldt Transit Authority,13
57,Modoc Transportation Agency,13
39,Eastern Sierra Transit Authority,12
58,Monterey-Salinas Transit,12
...,...,...
50,Klamath Trinity Non-Emergency Transportation​,3
63,Palo Verde Valley Transit Agency,3
14,City of Fairfield,2
53,Livermore Amador Valley Transit Authority,2
